### Method 3: ExplainSumm by Summarize Quality // Añadir esta segmentación a EvalSummMetrics

In [3]:
import time
import shap
import optuna 

import numpy as np
import pandas as pd

from tqdm.notebook import tqdm 

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from src.explainability import ExplainSumm
from src.data_visualization import visualize_metrics
from src.utils import train_test_split_comparision

from data.policy_data import policy_dict

from IPython.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
pd.set_option('display.max_columns', None)

In [36]:
# 1. Load processed datasets
df = pd.concat(
    [
        pd.read_pickle("data/processed/fe_train_comparisons.pkl"),
        pd.read_pickle("data/processed/fe_validation_comparisons.pkl"),
    ], 
    ignore_index=True
)

# 2. Non-features definition
comparison_columns = [
    "worker",
    "batch",
    "split",
    "id",
    "source",
    "subsource",
    "title",
    "text",
    "summary_0",
    "policy_0",
    "note_0",
    "summary_1",
    "policy_1",
    "note_1",
    "confidence",
    "ref_summary",
    "choice",
]

# 3. Target
target = ["choice"]

In [33]:
# Policy Evaluation
policy_eval = pd.read_csv("data/policy_eval.csv")

# Policy data
policy_data = pd.DataFrame(
    policy_dict.values(),
    index=policy_dict.keys(),
    columns=["Policy Type", "Parameters"],
)

policy_data["Parameters"] = policy_data["Parameters"].apply(
    lambda x: float(x.replace("B", "")))

# Policy DataFrame
merged_df = (
    policy_data.reset_index()
    .merge(policy_eval, left_on=["Policy Type", "Parameters"], right_on=["Model", "Parameters (B)"], how="left")
    .drop(columns=["Model", "Parameters (B)"])
    .rename(columns={"index": "Policy", "Parameters": "Parameters (B)"})
)

In [37]:
merged_df

,Policy,Policy Type,Parameters (B),Fraction preferred to ref,Quality
0,sup2,Supervised,1.30,0.261815,Low
1,sup1,Supervised,0.75,0.189002,Low
2,sup4_6b_t0.7,Supervised,6.70,0.323605,Low
3,sup4_ppo_rm3_kl10,RL policy,1.30,0.409953,Medium
4,sup4_ppo_rm3_kl20,RL policy,1.30,0.409953,Medium
5,sup4_t0.7,Supervised,1.30,0.261815,Low
6,sup3_6b,Supervised,6.70,0.323605,Low
7,sup2_bo8_rm1,BoN policy,1.30,0.429650,Medium
8,sup4_6b_ppo_rm4_6b_kl14,RL policy,6.70,0.660460,High
9,sup4_6b_ppo_rm3_6b_kl15,RL policy,6.70,0.660460,High


In [ ]:
# # create a condition for rows to be dropped
# condition = (df["policy_0"] == "ref") | (df["policy_1"] == "ref")

# # keep only those rows where condition is False
# df = df[condition]
# df.reset_index(drop=True)

In [38]:
df

,worker,batch,split,id,source,subsource,title,text,summary_0,policy_0,note_0,summary_1,policy_1,note_1,confidence,choice,ref_summary,m0_rouge_1_f,m0_rouge_2_f,m0_rouge_l_f,m1_rouge_1_f,m1_rouge_2_f,m1_rouge_l_f,m0_bleu,m1_bleu,m0_flesch_reading_ease,m0_syllable_count,m0_lexicon_count,m0_sentence_count,m0_char_count,m0_letter_count,m0_polysyllab_count,m0_monosyllab_count,m1_flesch_reading_ease,m1_syllable_count,m1_lexicon_count,m1_sentence_count,m1_char_count,m1_letter_count,m1_polysyllab_count,m1_monosyllab_count,m0_compression_ratio,m0_jaccard_similarity_1,m0_jaccard_similarity_2,m1_compression_ratio,m1_jaccard_similarity_1,m1_jaccard_similarity_2,m0_text_summary_xfmr_similarity,m1_text_summary_xfmr_similarity,m0_ref_summary_xfmr_similarity,m1_ref_summary_xfmr_similarity
0,qo6WIyEh27cwAjWpA3Q60J7NaDxzQJ,batch3,train,t3_34xale,reddit,relationships,Mother not speaking to me because of a trip ...,My boyfriend and I are long distance. We have ...,Mum is mad at me for not flying on my own trip...,sup1,NaN,I have made sure my mother is comfortable with...,sup1,NaN,NaN,1,mum isn't speaking to me because I booked a fl...,0.363636,0.181818,0.303030,0.368421,0.100000,0.210526,0.057345,0.046340,97.54,17.0,16.0,1.0,53.0,52.0,0.0,15.0,71.48,32.0,25.0,1.0,102.0,101.0,2.0,20.0,0.051750,0.107914,0.030418,0.095890,0.111888,0.029630,0.715529,0.696612,0.770994,0.665529
1,qo6WIyEh27cwAjWpA3Q60J7NaDxzQJ,batch3,train,t3_34xale,reddit,relationships,Mother not speaking to me because of a trip ...,My boyfriend and I are long distance. We have ...,I have made sure my mother is comfortable with...,sup1,NaN,mum isn't speaking to me because I booked a fl...,ref,NaN,NaN,1,mum isn't speaking to me because I booked a fl...,0.368421,0.100000,0.210526,1.000000,1.000000,1.000000,0.046340,1.000000,71.48,32.0,25.0,1.0,102.0,101.0,2.0,20.0,77.57,24.0,19.0,1.0,74.0,71.0,0.0,14.0,0.095890,0.111888,0.029630,0.070015,0.136691,0.045455,0.696612,0.640240,0.665529,1.000000
2,qo6WIyEh27cwAjWpA3Q60J7NaDxzQJ,batch3,train,t3_34xale,reddit,relationships,Mother not speaking to me because of a trip ...,My boyfriend and I are long distance. We have ...,mum isn't speaking to me because I booked a fl...,ref,NaN,Mum thought I was going to road trip with my b...,sup1,NaN,NaN,0,mum isn't speaking to me because I booked a fl...,1.000000,1.000000,1.000000,0.325581,0.000000,0.279070,1.000000,0.008724,77.57,24.0,19.0,1.0,74.0,71.0,0.0,14.0,97.20,38.0,32.0,4.0,136.0,132.0,1.0,27.0,0.070015,0.136691,0.045455,0.127093,0.154930,0.039568,0.640240,0.727418,1.000000,0.793905
3,qo6WIyEh27cwAjWpA3Q60J7NaDxzQJ,batch3,train,t3_34xale,reddit,relationships,Mother not speaking to me because of a trip ...,My boyfriend and I are long distance. We have ...,Mum thought I was going to road trip with my b...,sup1,NaN,Mum is mad at me for not flying on my own trip...,sup1,NaN,NaN,0,mum isn't speaking to me because I booked a fl...,0.325581,0.000000,0.279070,0.363636,0.181818,0.303030,0.008724,0.057345,97.20,38.0,32.0,4.0,136.0,132.0,1.0,27.0,97.54,17.0,16.0,1.0,53.0,52.0,0.0,15.0,0.127093,0.154930,0.039568,0.051750,0.107914,0.030418,0.727418,0.715529,0.793905,0.770994
4,LjvoXOAj5op3WqNnn5b7TZTG8mK7gM,batch3,train,t3_1zwek5,reddit,AskReddit,Can I sue my property management company and l...,My landlord left a falsified message taped to ...,My landlord is harassing me and my neighbours ...,sup1,NaN,landlord pretended to be another tenant and wr...,ref,NaN,NaN,1,landlord pretended to be another tenant and wr...,0.137931,0.000000,0.137931,1.000000,1.000000,1.000000,0.007511,1.000000,70.63,48.0,35.0,2.0,157.0,155.0,4.0,26.0,48.81,47.0,28.0,2.0,145.0,143.0,7.0,17.0,0.120353,0.129213,0.030100,0.108381,0.113636,0.037931,0.803380,0.603939,0.538846,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178859,soP7Okyp0e1WdYxQC9IldpwnznT6V1,edit_b2_eval_test,v

In [52]:
merged_df.loc[merged_df["Quality"] == "Low", "Policy"]

0                 sup2
1                 sup1
2         sup4_6b_t0.7
5            sup4_t0.7
6              sup3_6b
10    pretrain_12b_t.5
16    supcnndm3_6b_t.3
17             sup4_6b
31                sup3
32                sup4
33     pretrain_6b_t.7
34         pretrain_6b
35        pretrain_12b
40         sup4_6b_t.3
41          sup4_6b_t1
42         sup4_6b_t.5
45         sup4_6b_p.8
46         sup4_6b_p.7
47         sup4_6b_p.9
Name: Policy, dtype: object

In [53]:
# create a condition for rows to be dropped
low_condition = (
    df["policy_0"].isin(
        merged_df.loc[merged_df["Quality"] == "Low", "Policy"]
    ) &
    df["policy_1"].isin(
        merged_df.loc[merged_df["Quality"] == "Low", "Policy"]
    ) 
)

df.loc[low_condition, ["policy_0", "policy_1"]]

,policy_0,policy_1
0,sup1,sup1
3,sup1,sup1
5,sup1,sup1
7,sup1,sup1
8,sup1,sup1
...,...,...
178854,supcnndm3_6b_t.3,supcnndm3_6b_t.3
178855,supcnndm3_6b_t.3,supcnndm3_6b_t.3
178856,supcnndm3_6b_t.3,supcnndm3_6b_t.3
178857,supcnndm3_6b_t.3,supcnndm3_6b_t.3


In [54]:
# create a condition for rows to be dropped
medium_condition = (
    df["policy_0"].isin(
        merged_df.loc[merged_df["Quality"] == "Medium", "Policy"]
    ) &
    df["policy_1"].isin(
        merged_df.loc[merged_df["Quality"] == "Medium", "Policy"]
    ) 
)

df.loc[medium_condition, ["policy_0", "policy_1"]]

,policy_0,policy_1
48112,sup4_ppo_rm3_kl10,sup4_ppo_rm3_kl10
48113,sup4_ppo_rm3_kl10,sup4_ppo_rm3_kl10
48114,sup4_ppo_rm3_kl10,sup4_ppo_rm3_kl10
48115,sup4_ppo_rm3_kl10,sup4_ppo_rm3_kl10
48116,sup4_ppo_rm3_kl10,sup4_ppo_rm3_kl10
...,...,...
176578,sup4_ppo_rm3_kl10,sup4_bo64_rm3
176581,sup4_ppo_rm3_kl10,sup4_ppo_rm3_kl10
176583,sup4_bo64_rm3,sup4_ppo_rm3_kl10
176586,sup4_ppo_rm3_kl10,sup4_ppo_rm3_kl10


In [55]:
# create a condition for rows to be dropped
high_condition = (
    df["policy_0"].isin(
        merged_df.loc[merged_df["Quality"] == "High", "Policy"]
    ) &
    df["policy_1"].isin(
        merged_df.loc[merged_df["Quality"] == "High", "Policy"]
    ) 
)

df.loc[high_condition, ["policy_0", "policy_1"]]

,policy_0,policy_1
84296,sup4_bo128_rm3_6b,sup4_bo128_rm3_6b
84565,sup4_bo128_rm3_6b,sup4_bo128_rm3_6b
84669,sup4_bo128_rm3_6b,sup4_bo128_rm3_6b
84703,sup4_bo128_rm3_6b,sup4_bo128_rm3_6b
84714,sup4_bo128_rm3_6b,sup4_bo128_rm3_6b
...,...,...
173973,sup4_bo128_rm3_6b,sup4_bo128_rm3_6b
173980,sup4_bo128_rm3_6b,sup4_bo128_rm3_6b
174033,sup4_bo128_rm3_6b,sup4_bo128_rm3_6b
174036,sup4_bo128_rm3_6b,sup4_bo128_rm3_6b
